In [1]:
from tensorflow.keras import layers, Model
from tensorflow.python.client import device_lib
# Encoder
input_img = layers.Input(shape=(64, 64, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2))(x)

num_styles = 100
# Style Embedding
style_input = layers.Input(shape=(num_styles,))
style_embedding = layers.Dense(128, activation='relu')(style_input)

# Decoder
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# 모델 생성
model = Model([input_img, style_input], decoded)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

NotFoundError: dlopen(/Users/kimeunsur/yes/envs/newenv/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Users/kimeunsur/yes/envs/newenv/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <2814A58E-D752-317B-8040-131217E2F9AA> /Users/kimeunsur/yes/envs/newenv/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# 가상의 데이터 생성
num_samples = 1000  # 데이터 샘플 수
X_gothic = np.random.rand(num_samples, 64, 64, 1)  # 고딕체 이미지 (입력)
Y_target = np.random.rand(num_samples, 64, 64, 1)  # 타겟 폰트 이미지 (출력)

# 스타일 레이블 생성
style_labels = np.random.randint(0, num_styles, size=(num_samples,))
style_one_hot = to_categorical(style_labels, num_classes=num_styles)  # 원-핫 인코딩

# 정규화 (픽셀 값을 [0, 1]로 스케일링)
X_gothic = X_gothic / 255.0
Y_target = Y_target / 255.0

In [ ]:
# 모델 학습
history = model.fit(
    [X_gothic, style_one_hot],  # 입력: 고딕체 이미지와 스타일 정보
    Y_target,                   # 출력: 타겟 폰트 이미지
    validation_split=0.2,       # 검증 데이터 비율
    epochs=50,                  # 학습 반복 횟수
    batch_size=32               # 배치 크기
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()